# Modelado de predicción de aceptación de opción de pago

### By: Sergio Quintero

#### Objetivo: Desarrollar un modelo que permita predecir si el cliente aceptará una opción de pago a partir de los datos demográficos, financieros, y de negocio

## 1. Exploración y limpieza de los datos

#### Importación de los datos

In [1]:
import pandas as pd

datos_train = pd.read_csv('data/prueba_op_base_pivot_var_rpta_alt_enmascarado_trtest.csv')
datos_pagos = pd.read_csv('data/prueba_op_maestra_cuotas_pagos_mes_hist_enmascarado_completa.csv')
datos_probabilidades = pd.read_csv('data/prueba_op_probabilidad_oblig_base_hist_enmascarado_completa.csv')
datos_demograficos = pd.read_csv('data/prueba_op_master_customer_data_enmascarado_completa.csv')
datos_predecir = pd.read_csv('data/prueba_op_base_pivot_var_rpta_alt_enmascarado_oot.csv')

Remover duplicados

In [2]:
datos_demograficos_sorted = datos_demograficos.sort_values(by=['nit_enmascarado', 'year', 'month'], ascending=[True, False, False])
datos_demograficos_unicos = datos_demograficos_sorted.drop_duplicates(subset=['nit_enmascarado'], keep='first')

In [3]:
duplicados_demograficos_unicos = datos_demograficos_unicos[datos_demograficos_unicos.duplicated(subset=['nit_enmascarado'], keep=False)]
print(f'Duplicados en datos_demograficos: {len(duplicados_demograficos_unicos)}')

Duplicados en datos_demograficos: 0


Merge datos entrenamiento con datos demograficos

In [4]:
dataset = pd.merge(datos_train, datos_demograficos_unicos, on='nit_enmascarado', how='inner')
dataset.shape

(456290, 85)

Concatenar nit con num_obligacion

In [5]:
dataset['nit_num_oblig_id'] = dataset['nit_enmascarado'].astype(str) + '#' + dataset['num_oblig_orig_enmascarado'].astype(str) + '#' + dataset['num_oblig_enmascarado'].astype(str)
dataset['nit_num_oblig_id'].head()

0    630611#219718#863073
1     59412#789567#290775
2    277595#1045909#34433
3     24588#1061389#18953
4    431682#820319#260023
Name: nit_num_oblig_id, dtype: object

In [6]:
dataset['nit_num_oblig'] = dataset['nit_enmascarado'].astype(str) + '#' + dataset['num_oblig_enmascarado'].astype(str)
dataset['nit_num_oblig'].head()

0    630611#863073
1     59412#290775
2     277595#34433
3      24588#18953
4    431682#260023
Name: nit_num_oblig, dtype: object

Datos probabilidades

In [7]:
datos_probabilidades['nit_num_oblig'] = datos_probabilidades['nit_enmascarado'].astype(str) + '#' + datos_probabilidades['num_oblig_enmascarado'].astype(str)
datos_probabilidades['nit_num_oblig'].head()

0    296482#102381
1    391957#742315
2    229894#359919
3    478963#239064
4    349609#923348
Name: nit_num_oblig, dtype: object

In [8]:
datos_probabilidades = pd.merge(datos_probabilidades, dataset[['nit_num_oblig', 'fecha_var_rpta_alt']], on='nit_num_oblig', how='inner')
datos_probabilidades['fecha_var_rpta_alt'].head()

0    202311
1    202309
2    202309
3    202310
4    202309
Name: fecha_var_rpta_alt, dtype: int64

In [9]:
datos_probabilidades.head()

,nit_enmascarado,num_oblig_enmascarado,fecha_corte,lote,prob_propension,prob_alrt_temprana,prob_auto_cura,nit_num_oblig,fecha_var_rpta_alt
0,296482,102381,202308,1,0.761350,0.193744,0.684784,296482#102381,202311
1,229894,359919,202307,1,0.835373,0.285157,0.826225,229894#359919,202309
2,102595,857453,202307,1,0.742234,0.364649,0.527945,102595#857453,202309
3,508610,638552,202312,2,0.382216,0.606773,0.176670,508610#638552,202310
4,307136,561298,202306,1,0.806566,0.765648,0.533541,307136#561298,202309


Eliminar datos duplicados probabilidades

In [10]:
datos_probabilidades_filtrados = datos_probabilidades[datos_probabilidades['fecha_corte'] <= datos_probabilidades['fecha_var_rpta_alt']]
datos_probabilidades_filtrados_sorted = datos_probabilidades_filtrados.sort_values(by=['nit_num_oblig', 'fecha_corte'], ascending=[True, False])
datos_probabilidades_final = datos_probabilidades_filtrados_sorted.drop_duplicates(subset=['nit_num_oblig'], keep='first')

In [11]:
datos_probabilidades_final.head()

,nit_enmascarado,num_oblig_enmascarado,fecha_corte,lote,prob_propension,prob_alrt_temprana,prob_auto_cura,nit_num_oblig,fecha_var_rpta_alt
2574258,1,104488,202308,2,0.402569,0.682365,0.185597,1#104488,202308
2871968,100,362910,202308,1,0.232395,0.739224,0.182060,100#362910,202308
2336247,100,362960,202310,2,0.435284,0.737880,0.187280,100#362960,202310
3901730,1000,1066337,202310,1,0.831019,0.414249,0.340294,1000#1066337,202310
2542791,100002,45269,202309,1,0.281339,0.630578,0.294072,100002#45269,202309


In [12]:
duplicados_datos_probabilidades_unicos = datos_probabilidades_final[datos_probabilidades_final.duplicated(subset=['nit_num_oblig'], keep=False)]
print(f'Duplicados en datos_probabilidades: {len(duplicados_datos_probabilidades_unicos)}')

Duplicados en datos_probabilidades: 0


In [13]:
datos_probabilidades_final.drop('fecha_var_rpta_alt', axis=1, inplace=True)

/tmp/ipykernel_1485/173397831.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_probabilidades_final.drop('fecha_var_rpta_alt', axis=1, inplace=True)


Merge dataset con datos de probabilidades

In [14]:
dataset = pd.merge(dataset, datos_probabilidades_final, on='nit_num_oblig', how='inner')
dataset.shape

(456233, 94)

Datos pagos

In [15]:
datos_pagos['nit_num_oblig'] = datos_pagos['nit_enmascarado'].astype(str) + '#' + datos_pagos['num_oblig_enmascarado'].astype(str)
datos_pagos['nit_num_oblig'].head()

0    482906#362297
1    121735#186855
2    582719#675503
3    299903#107931
4     88625#566060
Name: nit_num_oblig, dtype: object

In [16]:
datos_pagos_sorted = datos_pagos.sort_values(by=['nit_num_oblig', 'fecha_corte'], ascending=[True, False])
datos_pagos_unicos = datos_pagos_sorted.drop_duplicates(subset=['nit_num_oblig'], keep='first')

In [17]:
duplicados_datos_pagos_unicos = datos_pagos_unicos[datos_pagos_unicos.duplicated(subset=['nit_num_oblig'], keep=False)]
print(f'Duplicados en datos_pagos: {len(duplicados_datos_pagos_unicos)}')

Duplicados en datos_pagos: 0


In [21]:
df = pd.merge(dataset, datos_pagos_unicos, on='nit_num_oblig', how='inner')
df.shape

(456233, 107)

Depuración del DataSet

Se depuraron los datos que puedan tener una relación directa con la variable a predecir, la cual no sería posible conocer su valor a priori

In [23]:
columns_to_delete = ['marca_alt_apli','descripcion_ranking_mejor_ult','marca_alt_rank','pagos_tanque','descripcion_ranking_mejor_ult','sector','segm','act_econom','cant_acuerdo_binario','cant_gestiones_binario',
                     'nit_enmascarado_x','num_oblig_orig_enmascarado', 'num_oblig_enmascarado_x',
                     'producto_cons','alter_posible2_2','alter_posible3_2',
                     'marca_alt_rank','marca_alternativa','marca_alternativa_orig','ano_nac_cli',
                     'nit_enmascarado_y','num_oblig_enmascarado_y','fecha_corte_x','nit_enmascarado',
                     'num_oblig_enmascarado','fecha_corte_y','fecha_pago_minima','fecha_pago_maxima',
                     'nombre_dpto_dirp', 'fecha_var_rpta_alt', 'cod_tipo_doc', 'f_ult_mantenimiento', 
                     'year', 'month', 'ingestion_day', 'tipo_var_rpta_alt', 'desc_alternativa1','desc_alternativa2',
                     'desc_alternativa3', 'smmlv']

In [24]:
df.drop(columns_to_delete, axis=1, inplace=True)


Eliminar duplicados datos pagos

Merge dataset con datos de pagos

In [25]:
df.shape

(456233, 69)

Se analizarán las variables con mayor cantidad de nulos para ver si se elimina la variable o se pueden imputar los datos faltantes con algún estimador insesgado

In [26]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
missing_values

tipo_vivienda               307004
alternativa_aplicada_agr    291288
marca_agrupada_rgo          291287
nivel_academico             252083
porc_pago_mes               247059
nicho                       240156
canal_actualizacion          44232
estado_civil                 34984
ciiu                         29554
subsector                    29554
ocup                         24119
cant_acuerdo                 15026
cant_gestiones               14951
subsegm                      13262
genero_cli                   11796
personas_dependientes        11783
edad_cli                     11782
num_hijos                    11782
porc_pago                     7913
origen_fondos                 4564
vlr_obligacion                 486
declarante                     148
rpc                              2
dtype: int64

Variables a eliminar por su cantidad de nulos

In [27]:
columns_to_delete = ['tipo_vivienda','alternativa_aplicada_agr','marca_agrupada_rgo','nivel_academico',
                     'porc_pago_mes','nicho','canal_actualizacion','estado_civil']
df.drop(columns_to_delete, axis=1, inplace=True)

Imputación de datos faltantes

In [28]:
ciiu_counts = df['ciiu'].value_counts()
print(len(ciiu_counts))
ciiu_counts

456


ciiu
ASALARIADOS                                                             356611
RENTISTAS DE CAPITAL SÓLO PARA PERSONAS NATURALES                         6192
OTRAS ACTIVIDADES DE SERVICIO DE APOYO A LAS EMPRESAS N.C.P.              4817
TRANSPORTE DE CARGA POR CARRETERA                                         3082
OTRAS ACTIVIDADES DE SERVICIOS PERSONALES N.C.P.                          1982
                                                                         ...  
ACTIVIDADES DE LAS CASAS DE CAMBIO                                           1
EXTRACCIÓN DE OTROS MINERALES NO METÁLICOS N.C.P.                            1
FABRICACIÓN DE INSTRUMENTOS ÓPTICOS Y EQUIPO FOTOGRÁFICO                     1
ACTIVIDADES DE TELECOMUNICACIÓN SATELITAL                                    1
CAZA ORDINARIA Y MEDIANTE TRAMPAS Y ACTIVIDADES DE SERVICIOS CONEXAS         1
Name: count, Length: 456, dtype: int64

In [29]:
df['ciiu'].fillna('NO REGISTRA', inplace=True)

/tmp/ipykernel_1485/2164114025.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ciiu'].fillna('NO REGISTRA', inplace=True)


In [30]:
df['ciiu'] = ['ASALARIADO' if valor == 'ASALARIADOS' else 'NO REGISTRA' if valor == 'NO REGISTRA' else 'OTRO' for valor in df['ciiu']]

In [31]:
subsector_counts = df['subsector'].value_counts()
print(len(subsector_counts))
subsector_counts

85


subsector
PERSONAS NATURALES                    362803
SERVICIOS A EMPRESAS                   16272
SERVICIOS A PERSONAS                    7228
COMERCIO DE VARIEDADES Y VESTUARIO      5914
TRANSPORTE TERRESTRE                    4614
                                       ...  
BEBIDAS ALCOHÓLICAS                        5
TABACO                                     4
ALGODÓN                                    4
EPS Y SALUD PREPAGADA (SALUD)              2
BANCOS                                     2
Name: count, Length: 85, dtype: int64

In [32]:
df['subsector'].fillna('NO REGISTRA', inplace=True)

/tmp/ipykernel_1485/3864922962.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['subsector'].fillna('NO REGISTRA', inplace=True)


In [33]:
df['subsector'] = ['NATURALES' if valor == 'PERSONAS NATURALES' else 'NO REGISTRA' if valor == 'NO REGISTRA' else 'OTRO' for valor in df['subsector']]

In [34]:
ocupacion_counts = df['ocup'].value_counts()
print(len(ocupacion_counts))
ocupacion_counts

14


ocup
EMPLEADO                     286575
INDEPENDIENTE                 33388
ESTUDIANTE                    27198
PENSIONADO                    22347
PROFESIONAL INDEPENDIENTE     18458
AMA DE CASA                   13519
COMERCIANTE                    8072
OTRA                           7532
SOCIO O EMPLEADO - SOCIO       6265
RENTISTA DE CAPITAL            2452
DESEMPLEADO CON INGRESOS       1861
AGRICULTOR                     1848
DESEMPLEADO SIN INGRESOS       1373
GANADERO                       1226
Name: count, dtype: int64

In [35]:
df['ocup'].fillna('NO REGISTRA', inplace=True)

/tmp/ipykernel_1485/3233819139.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ocup'].fillna('NO REGISTRA', inplace=True)


In [36]:
df['cant_acuerdo'].describe()

count    441207.000000
mean          1.110508
std           2.125219
min           0.000000
25%           0.000000
50%           0.000000
75%           2.000000
max          40.000000
Name: cant_acuerdo, dtype: float64

In [37]:
df['cant_acuerdo'].fillna(0, inplace=True)

/tmp/ipykernel_1485/1987181876.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['cant_acuerdo'].fillna(0, inplace=True)


In [38]:
df['cant_gestiones'].fillna(0, inplace=True)

/tmp/ipykernel_1485/2295259971.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['cant_gestiones'].fillna(0, inplace=True)


In [39]:
subsegmento_counts = df['subsegm'].value_counts()
print(len(subsegmento_counts))
subsegmento_counts

16


subsegm
MEDIO                    221426
BASICO                   101419
ALTO                      56204
PEQUE#O                   27126
MEDIANO                   11504
PEQUENA                    8053
GRANDE                     7357
MEDIANA                    2999
MiPyme                     2875
PLUS                       1696
RELACIONAMIENTO            1245
PREFERENCIAL PLUS           712
PREF CONCILIACION           223
PREFERENCIAL COLOMBIA       119
NO APLICA                     8
PREF.COLOMBIA                 5
Name: count, dtype: int64

In [40]:
df['subsegm'].fillna('NO REGISTRA', inplace=True)

/tmp/ipykernel_1485/2440278629.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['subsegm'].fillna('NO REGISTRA', inplace=True)


In [41]:
df['subsegm'] = ['MEDIO' if valor == 'MEDIANO' or valor == 'MEDIANA' else valor for valor in df['subsegm']]
df['subsegm'] = ['PEQUENO' if valor == 'PEQUE#O' or valor == 'PEQUENA' else valor for valor in df['subsegm']]
df['subsegm'] = ['ALTO' if valor == 'GRANDE' else valor for valor in df['subsegm']]

In [42]:
generocli_counts = df['genero_cli'].value_counts()
print(len(generocli_counts))
generocli_counts

2


genero_cli
M    231425
F    213012
Name: count, dtype: int64

In [43]:
df['genero_cli'].fillna('NO REGISTRA', inplace=True)

/tmp/ipykernel_1485/2358772325.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['genero_cli'].fillna('NO REGISTRA', inplace=True)


In [44]:
personasdepend_counts = df['personas_dependientes'].value_counts()
print(len(personasdepend_counts))
personasdepend_counts

26


personas_dependientes
0.0     387037
1.0      22632
2.0      19171
3.0       9596
4.0       4121
5.0       1302
6.0        302
7.0        141
8.0         51
11.0        15
10.0        15
9.0         13
60.0        10
16.0         9
15.0         6
20.0         6
30.0         4
22.0         3
32.0         3
12.0         3
21.0         3
13.0         3
91.0         1
40.0         1
31.0         1
24.0         1
Name: count, dtype: int64

In [45]:
df['personas_dependientes'].describe()

count    444450.000000
mean          0.265395
std           0.877377
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          91.000000
Name: personas_dependientes, dtype: float64

In [46]:
df['personas_dependientes'].fillna(0, inplace=True)

/tmp/ipykernel_1485/2320525710.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['personas_dependientes'].fillna(0, inplace=True)


In [47]:
df['personas_dependientes'].describe()

count    456233.000000
mean          0.258541
std           0.866995
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          91.000000
Name: personas_dependientes, dtype: float64

In [48]:
df['num_hijos'].describe()

count    444451.000000
mean          0.221988
std           0.865770
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          96.000000
Name: num_hijos, dtype: float64

In [49]:
df['num_hijos'].fillna(0, inplace=True)

/tmp/ipykernel_1485/2290332444.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['num_hijos'].fillna(0, inplace=True)


In [50]:
df['edad_cli'].describe()

count    444451.000000
mean         41.187082
std          13.880988
min           0.000000
25%          31.000000
50%          39.000000
75%          49.000000
max         123.000000
Name: edad_cli, dtype: float64

In [51]:
df['num_hijos'].fillna(df['num_hijos'].mean(), inplace=True)

/tmp/ipykernel_1485/1249752402.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['num_hijos'].fillna(df['num_hijos'].mean(), inplace=True)


In [52]:
df['edad_cli'].describe()

count    444451.000000
mean         41.187082
std          13.880988
min           0.000000
25%          31.000000
50%          39.000000
75%          49.000000
max         123.000000
Name: edad_cli, dtype: float64

In [53]:
df['edad_cli'].fillna(df['edad_cli'].mean(), inplace=True)

/tmp/ipykernel_1485/3891237186.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['edad_cli'].fillna(df['edad_cli'].mean(), inplace=True)


In [54]:
df['porc_pago'].fillna(0, inplace=True)

/tmp/ipykernel_1485/608754133.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['porc_pago'].fillna(0, inplace=True)


In [55]:
origenfondos_counts = df['origen_fondos'].value_counts()
print(len(origenfondos_counts))
origenfondos_counts

10


origen_fondos
SALARIO Y DEMAS PAGOS LABORALES           242450
OTROS                                     185963
HONORARIOS Y COMISIONES                     9272
VENTAS NETAS                                5287
DESARROLLO DEL OBJETO SOCIAL DE LA PJ       4261
UTILIDADES DEL NEGOCIO                      1252
CAPITALIZACION POR PARTE DE LOS SOCIOS      1181
INTERESES Y RENDIMIENTOS FINANCIEROS        1129
DIVIDENDOS Y PARTICIPACIONES                 860
RENDIMIENTOS POR INVERSIÓN                    14
Name: count, dtype: int64

In [56]:
df['origen_fondos'].fillna("NO REGISTRA", inplace=True)

/tmp/ipykernel_1485/56577716.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['origen_fondos'].fillna("NO REGISTRA", inplace=True)


In [57]:
df['vlr_obligacion'].describe()

count    4.557470e+05
mean     1.169339e+07
std      2.502361e+07
min      0.000000e+00
25%      1.580381e+06
50%      4.177424e+06
75%      1.083460e+07
max      1.589994e+09
Name: vlr_obligacion, dtype: float64

In [58]:
df['vlr_obligacion'].fillna(df['vlr_obligacion'].median(), inplace=True)

/tmp/ipykernel_1485/2923847177.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['vlr_obligacion'].fillna(df['vlr_obligacion'].median(), inplace=True)


In [59]:
declarante_counts = df['declarante'].value_counts()
print(len(declarante_counts))
declarante_counts

2


declarante
N    408339
S     47746
Name: count, dtype: int64

In [60]:
df['declarante'].fillna('N', inplace=True)

/tmp/ipykernel_1485/1367438067.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['declarante'].fillna('N', inplace=True)


In [61]:
df['rpc'].fillna(0, inplace=True)

/tmp/ipykernel_1485/307850175.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['rpc'].fillna(0, inplace=True)


In [62]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
missing_values

Series([], dtype: int64)

In [63]:
int_columns = df.select_dtypes(include=['int64']).columns
int_columns

Index(['var_rpta_alt', 'min_mora', 'max_mora', 'dias_mora_fin',
       'cant_alter_posibles', 'promesas_cumplidas',
       'cant_promesas_cumplidas_binario', 'f_vinc', 'lote'],
      dtype='object')

In [64]:
df['min_mora'].describe()

count    456233.000000
mean         26.836246
std          25.835780
min           1.000000
25%           4.000000
50%          17.000000
75%          46.000000
max          90.000000
Name: min_mora, dtype: float64

In [65]:
df['max_mora'].describe()

count    456233.000000
mean         48.530749
std          32.117126
min           1.000000
25%          23.000000
50%          43.000000
75%          70.000000
max        1612.000000
Name: max_mora, dtype: float64

In [66]:
df['max_mora'].quantile(0.95)

np.float64(109.0)

In [67]:
df['max_mora'] = [109 if value > 109 else value for value in df['max_mora']]

In [68]:
df['dias_mora_fin'].describe()


count    456233.000000
mean         37.244879
std          35.205280
min           0.000000
25%           0.000000
50%          29.000000
75%          60.000000
max         609.000000
Name: dias_mora_fin, dtype: float64

In [69]:
df['dias_mora_fin'].quantile(0.95)

np.float64(106.0)

In [70]:
df['max_mora'] = [106 if value > 106 else value for value in df['dias_mora_fin']]

In [71]:
df['cant_alter_posibles'].describe()

count    456233.000000
mean          1.810242
std           0.864049
min           1.000000
25%           1.000000
50%           2.000000
75%           3.000000
max          12.000000
Name: cant_alter_posibles, dtype: float64

In [72]:
df['promesas_cumplidas'].describe()

count    456233.000000
mean          0.211475
std           0.688404
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          16.000000
Name: promesas_cumplidas, dtype: float64

In [73]:
df['f_vinc'].head()

0    20200629
1    20180829
2    19870810
3    19850301
4    20080826
Name: f_vinc, dtype: int64

In [74]:
df['fecha_vinc'] = pd.to_datetime(df['f_vinc'].astype(str), format='%Y%m%d')

In [75]:
fecha_referencia = pd.to_datetime('2024-01-01')
df['antiguedad'] = (fecha_referencia - df['fecha_vinc']).dt.days

In [76]:
df.drop('fecha_vinc', axis=1, inplace=True)

In [77]:
df.drop('f_vinc', axis=1, inplace=True)

In [78]:
df['lote'] = df['lote'].astype(str)

In [79]:
lote_counts = df['lote'].value_counts()
print(len(lote_counts))
lote_counts

3


lote
1    221078
2    158646
3     76509
Name: count, dtype: int64

In [80]:
df['var_rpta_alt'] = df['var_rpta_alt'].astype(str)

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456233 entries, 0 to 456232
Data columns (total 61 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   var_rpta_alt                     456233 non-null  object 
 1   banca                            456233 non-null  object 
 2   segmento_x                       456233 non-null  object 
 3   producto_x                       456233 non-null  object 
 4   aplicativo_x                     456233 non-null  object 
 5   min_mora                         456233 non-null  int64  
 6   max_mora                         456233 non-null  int64  
 7   dias_mora_fin                    456233 non-null  int64  
 8   rango_mora                       456233 non-null  object 
 9   vlr_obligacion                   456233 non-null  float64
 10  vlr_vencido                      456233 non-null  float64
 11  saldo_capital                    456233 non-null  float64
 12  en

In [82]:
df.isnull().sum()

var_rpta_alt     0
banca            0
segmento_x       0
producto_x       0
aplicativo_x     0
                ..
pago_total       0
porc_pago        0
marca_pago_y     0
ajustes_banco    0
antiguedad       0
Length: 61, dtype: int64

In [83]:
df.shape

(456233, 61)

In [84]:
segmentox_counts = df['segmento_x'].value_counts()
print(len(segmentox_counts))
segmentox_counts

7


segmento_x
Personal         291978
Personal plus     87348
Micropyme         45303
Pymes             15188
Social            14093
Preferencial       2320
Empresarial           3
Name: count, dtype: int64

In [85]:
producto_x_counts = df['producto_x'].value_counts()
print(len(producto_x_counts))
producto_x_counts

24


producto_x
TARJETA DE CREDITO       195986
LIBRE INVERSION          161286
ROTATIVOS                 64608
CARTERA ORDINARIA          9580
LIBRANZA                   8404
HIPOTECARIO VIVIENDA       8238
CARTERA MICROCREDITO       4801
CREDIPAGO                  1165
SOBREGIRO                  1015
LEASING HABITACIONAL        501
TARJETAS DE CREDITO         193
CREDITOS DE CONSUMO         122
TESORERIA                   109
CREDIAGIL                    67
CREDITO A LA MANO            32
Titularizada                 30
LEASING                      22
REESTRUCTURADO               21
VENTA DIGITAL                19
CUENTA CORRIENTE             14
OTROS HIPOTECARIO            11
LIBRANZA EX EMPLEADOS         5
MICROCREDITO                  3
CREDITO HIPOTECARIO           1
Name: count, dtype: int64

In [86]:
aplicativo_x_counts = df['aplicativo_x'].value_counts()
print(len(aplicativo_x_counts))
aplicativo_x_counts

7


aplicativo_x
L    250245
M     82227
V     67099
K     46853
4      8279
D      1029
3       501
Name: count, dtype: int64

Dataset depurado

In [87]:
df.to_csv('dataset_depurado.csv', index=False)